In [1]:
import os
import sys

import h5py

import matplotlib as mpl

def set_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

width = 234.8775 # in pt.
width, height = set_size(width, fraction=0.8) # in inch
print(width, height)

fontsize = 9
mpl.rcParams.update({
    "figure.figsize": (width, height), 
    "figure.facecolor": 'white', 
    "figure.edgecolor": 'white',
    "savefig.dpi": 360, 
    "figure.subplot.bottom": 0.5, 
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": fontsize,
    "font.size": fontsize,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": fontsize-2,
    "xtick.labelsize": fontsize-2,
    "ytick.labelsize": fontsize-2,
    # tight layout,
    "figure.autolayout": True,
})

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from scipy.ndimage import uniform_filter1d
from scipy.stats import rankdata

2.6 1.6068883707497268


## Loading The Data

In [28]:
data_path = "/Users/romainegele/Documents/Argonne/deephyper-benchmark/lib/JAHSBench/build/jahs_bench_data"

import jahs_bench

In [36]:
dataset_list = [
    # "cifar10",
    # "colorectal_histology",
    "fashion_mnist"
]

for dataset in tqdm(dataset_list):

    random_state = 42
    rs = np.random.RandomState(random_state)
    num_models = 1_000
    
    bench = jahs_bench.Benchmark(
        task=dataset, 
        download=False, 
        save_dir=data_path,
        kind="surrogate",
    )

    M = []
    for i in tqdm(range(num_models)):
        config = bench.sample_config(rs)
        trajectory = bench(config, nepochs=200, full_trajectory=True)
        curve = [100-trajectory[i]["valid-acc"] for i in range(1, 201)]
        
        y = np.asarray(curve).reshape(1, -1)
        C = np.mean(y, axis=0).tolist()
        M.append(C)
    M = np.asarray(M)
    print(f"{M.shape=}")

    # ranking matrix per step
    R = rankdata(M, axis=0)
    print(f"{R.shape=}")

    idx_sorting = np.argsort(R[:,-1])
    
    # Stability of Ranking
    plt.figure()
    plt.imshow(R[idx_sorting], cmap="seismic", interpolation="nearest", aspect="auto")
    plt.colorbar(label="Rank")
    plt.xlabel("Training Epoch")
    plt.ylabel("Model's Index")
    plt.tight_layout()
    plt.savefig(f"jahs-plots/heatmap_ranks_jahs_{dataset}.png")
    plt.show()
    
    # Curves
    cmap = mpl.colormaps["seismic"]

    x = np.arange(1,201)
    fig = plt.figure()
    ax = plt.gca()

    for i in range(M.shape[0]):
        plt.plot(x, M[idx_sorting][i], color=cmap(i/M.shape[0]))

    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(1,200)
    plt.xlabel(r"Training Error")
    plt.ylabel("Validation Error")
    norm = mpl.cm.ScalarMappable(norm=None, cmap=cmap)
    norm.set_clim(1, 1000)
    cb = plt.colorbar(norm, ax=ax, label="Rank")
    plt.tight_layout()
    plt.savefig(f"jahs-plots/ranking_curves_jahs_{dataset}.png")
    plt.show()

  0%|          | 0/1 [00:00<?, ?it/s]

[17:13:11] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667849614592/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[17:13:11] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667849614592/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[17:13:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667849614592/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by

  0%|          | 0/1000 [00:00<?, ?it/s]

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [41]:
text = r"""
    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/lcbench/curves/ranking_curves_yahpo_XXX.png}
        \caption{YYY}
        \label{fig:ranking-curves-yahpo-XXX}
    \end{subfigure}"""

text = r"""
    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/yapho/curves/heatmap_ranks_yahpo_XXX.png}
        \caption{YYY}
        \label{fig:heatmap-yahpo-XXX}
    \end{subfigure}"""

In [42]:
for dataset in sorted(map_name2openmlid.keys()):
    print(text.replace("XXX", dataset).replace("YYY", dataset.replace("_", " ").replace("-", " ")), end="")


    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/yapho/curves/heatmap_ranks_yahpo_APSFailure.png}
        \caption{APSFailure}
        \label{fig:heatmap-yahpo-APSFailure}
    \end{subfigure}
    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/yapho/curves/heatmap_ranks_yahpo_Amazon_employee_access.png}
        \caption{Amazon employee access}
        \label{fig:heatmap-yahpo-Amazon_employee_access}
    \end{subfigure}
    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/yapho/curves/heatmap_ranks_yahpo_Australian.png}
        \caption{Australian}
        \label{fig:heatmap-yahpo-Australian}
    \end{subfigure}
    \begin{subfigure}[b]{0.24\textwidth}
        \centering
        \includegraphics[width=\textwidth]{figures/yapho/curves/heatmap_ranks_yahpo_Fashion-MNIST.png}
        \caption{Fashion MNIST}
      

In [42]:
len(dataset_list)

35